# K Nearest Neighbors Classifier

**Basic steps:**

1. Import the learning algorithm
2. Instantiate the model (choose hyper-parameters)
3. Learn the model
4. Predict the response

In [6]:
import pandas as pd


# Get Example Data

In [7]:
# target = InMichelin, whether or not a restaurant is in the Michelin guide

data = pd.read_csv("https://gattonweb.uky.edu/sheather/book/docs/datasets/MichelinNY.csv")
data.head()

,InMichelin,Restaurant Name,Food,Decor,Service,Price
0,0,14 Wall Street,19,20,19,50
1,0,212,17,17,16,43
2,0,26 Seats,23,17,21,35
3,1,44,19,23,16,52
4,0,A,23,12,19,24


In [9]:
data.columns != 'Restaurant Name'

array([ True, False,  True,  True,  True,  True])

In [10]:
# Delete extra variable that is not continuous
data = data.loc[:, data.columns != 'Restaurant Name']

data.head()

,InMichelin,Food,Decor,Service,Price
0,0,19,20,19,50
1,0,17,17,16,43
2,0,23,17,21,35
3,1,19,23,16,52
4,0,23,12,19,24


# Change variable names to X, y to create train/test split

In [13]:
y = data['InMichelin']
X = data.loc[:, data.columns != 'InMichelin']

print(y[0:5])
X.head()

0    0
1    0
2    0
3    1
4    0
Name: InMichelin, dtype: int64


,Food,Decor,Service,Price
0,19,20,19,50
1,17,17,16,43
2,23,17,21,35
3,19,23,16,52
4,23,12,19,24


# Train test split

In [16]:
from sklearn.model_selection import train_test_split

# Use train_test_split(X,y) to create four new data sets, defaults to .75/.25 split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

X_train.head()

,Food,Decor,Service,Price
27,17,19,17,41
65,18,17,15,37
130,20,17,20,41
77,20,20,18,51
76,27,26,27,95


### Train model with k=5

In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

#Print accuracy rounded to two digits to the right of decimal
print(knn.score(X_test, y_test))

knn.predict([[50,1,100,1]]) # y_pred includes your predictions

0.8048780487804879


array([0])

### Train model with k=10

In [25]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

#Print accuracy rounded to two digits to the right of decimal
print(knn.score(X_test, y_test))
y_pred = knn.predict(X_test)


0.9024390243902439


In [ ]:
y_pred # view predictions for test data

array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1])

## Using Cross validation for model evaluation

In [27]:
#import cross validation functions from sk learn

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# Set up function parameters for diff't cross validation strategies
kfold = KFold(n_splits=5)
skfold = StratifiedKFold(n_splits=5, shuffle=True)
rkf = RepeatedKFold(n_splits=5, n_repeats=10)

print("KFold: "+str(cross_val_score(KNeighborsClassifier(n_neighbors=1), X_train, y_train, cv=kfold).mean()))

print("StratifiedKFold:\n{}".format(
cross_val_score(KNeighborsClassifier(n_neighbors=5), X_train, y_train, cv=skfold).mean()))

print("RepeatedKFold:\n{}".format(
cross_val_score(KNeighborsClassifier(n_neighbors=5), X_train, y_train,  cv=rkf).mean()))


KFold: 0.6436666666666667
StratifiedKFold:
0.715179487179
RepeatedKFold:
0.7402


## Tuning models with grid search

In [ ]:
import numpy as np

np.arange(1, 15, 2)

array([ 1,  3,  5,  7,  9, 11, 13])

In [28]:
from sklearn.model_selection import GridSearchCV
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

#create dictionary data object with keys equal to parameter name 'n_neighbors' 
#for knn model and values equal to range of k values to create models for

param_grid = {'n_neighbors': [1,3,5,7,9,10,11,13] }#np.arange creates sequence of numbers for each k value

grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=10)

#use meta model methods to fit score and predict model:
grid.fit(X_train, y_train)

#extract best score and parameter by calling objects "best_score_" and "best_params_"
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))
print("test-set score: {:.3f}".format(grid.score(X_test, y_test)))


best mean cross-validation score: 0.797
best parameters: {'n_neighbors': 11}
test-set score: 0.829


/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [ ]:
# view data with complete tuning results
results = pd.DataFrame(grid.cv_results_)
results


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_n_neighbors,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split7_test_score,split7_train_score,split8_test_score,split8_train_score,split9_test_score,split9_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.001874,0.001734,0.666667,1.000000,1,{u'n_neighbors': 1},8,0.692308,1.000000,0.461538,...,0.750000,1.000000,0.636364,1.000000,0.818182,1.000000,0.000100,0.000135,0.135180,0.000000
1,0.001942,0.001761,0.723577,0.850953,3,{u'n_neighbors': 3},7,0.615385,0.872727,0.615385,...,0.750000,0.837838,0.636364,0.857143,0.818182,0.848214,0.000398,0.000175,0.115391,0.012895
2,0.001914,0.001751,0.739837,0.820248,5,{u'n_neighbors': 5},6,0.615385,0.836364,0.615385,...,0.666667,0.846847,0.727273,0.803571,0.818182,0.821429,0.000263,0.000116,0.100455,0.013431
3,0.001815,0.001693,0.764228,0.813016,7,{u'n_neighbors': 7},3,0.692308,0.818182,0.692308,...,0.750000,0.783784,0.727273,0.821429,0.818182,0.821429,0.000066,0.000046,0.062823,0.015528
4,0.001867,0.001697,0.764228,0.803081,9,{u'n_neighbors': 9},3,0.692308,0.790909,0.692308,...,0.833333,0.783784,0.727273,0.803571,0.818182,0.812500,0.000144,0.000035,0.060714,0.013094
5,0.001862,0.001766,0.756098,0.801279,10,{u'n_neighbors': 10},5,0.615385,0.809091,0.692308,...,0.833333,0.783784,0.727273,0.794643,0.818182,0.812500,0.000094,0.000081,0.083848,0.014280
6,0.001874,0.001751,0.780488,0.815727,11,{u'n_neighbors': 11},1,0.692308,0.809091,0.692308,...,0.916667,0.792793,0.727273,0.821429,0.818182,0.812500,0.000097,0.000044,0.077004,0.010623
7,0.001839,0.001732,0.780488,0.794956,13,{u'n_neighbors': 13},1,0.692308,0.818182,0.692308,...,0.916667,0.792793,0.727273,0.803571,0.818182,0.785714,0.000052,0.000055,0.077004,0.011848
